# formata-texto
Esse script recebe arquivos .txt extraídos do site da Câmara dos Deputados e executa operações de limpeza e formatação usando regex.

#### Importação de pacotes

In [ ]:
#!pip install unidecode

In [1]:
import glob
import os
import pandas as pd
import re
from unidecode import unidecode

import os
import zipfile
import io

import random

In [2]:
# VARIÁVEL GLOBAL PARA TESTAR VÁRIOS ENCODINGS
encoding = 'UTF-8'

#### Ler arquivos externos

In [3]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    
    '''
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    return files

In [19]:
def read_file(file_list, zip_file = False):
    random.seed(2020)
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    Funciona para os discursos em plenário
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    print('Reading Files...')
    content = []
    name_file_list = []
    if zip_file:
        for f in file_list:#[:5]:
            #print('-->',f)
            with zipfile.ZipFile(f) as zip_file:
                print(f, len(zip_file.filelist))
                ## serão selecionados 30% do total de cada arquivo de forma aleatória
                sample_size = round(len(zip_file.filelist)*.3)
                for name_file in random.sample(zip_file.filelist,sample_size):
                    name_file_list.append(unidecode(name_file.filename))
                    
                    with io.TextIOWrapper(zip_file.open(name_file.filename), encoding=encoding) as arq:
                        content.append(arq.read())

    else:
        # Lê o conteúdo dos arquivos de texto na lista
        content = [ open(file, encoding = encoding).read() for file in file_list ]
        name_file_list = [ unidecode(file) for file in file_list ]

    return content, name_file_list

def make_df_plen(file_list, content):
    print('Making  DF plen...')
    session_date = []
    for file in info:
        result = re.search("\-([A-Z\s]+)\-", file)# info[108251])
        if result is None:
            session_date.append('none')
        else:
            session_date.append(result.group(1))
        
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'             : [ file for file in content ],
        'ORIGINAL_CONTENT' : [ item for item in file_list ],
        'CLEAN_CONTENT'    : [ unidecode(item) for item in file_list ],
        'SESSION_TYPE'     : session_date,#[ re.search("\-([A-Z\s]+)\-", file).group(1) for file in content ],
        'SESSION_DATE'     : [ re.search("\d{8}", file).group(0) for file in content ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")
    df["MONTH"] = df.SESSION_DATE.dt.month
    df["YEAR"] = df.SESSION_DATE.dt.year

    return df

#### Funções de formatação e busca usando regex

In [5]:
def find_speakers(string):
    
    '''
    Essa função detecta o padrão de texto
    que antecede a fala de um deputado e
    retorna um objeto match (via re.find_all).
    Ele é útil para detectar QUANTOS deputados
    falaram em determinada string textual.
    '''
    
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    return matches

In [6]:
def extract_full_quote(clean_string, original_string):
    
    '''
    Essa função extrai todas as falas de Jair Bolsonaro
    em uma determinada string. O pattern de regex encontra,
    primeiro, uma fala qualquer do Presidente. Então, pega
    tudo que está entre essa fala e a fala de outro parlamentar 
    ou o fim do arquivo. Isso é necessário porque há arquivos
    que misturam a fala de vários parlamentares, geralmente
    quando estão envolvidos em uma discussão.
    
    Para fazer essa operação, são passados textos sem caracteres
    especiais unicode. Depois de feita a captura, usamos os índices
    das matches no regex para extrair o mesmo pedaço de texto
    na string original.

    '''
    
    #if "O SR. PRESIDENTE (Jair Bolsonaro)" in clean_string:
     #   pattern = "O SR\. PRESIDENTE (\(Jair Bolsonaro\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
      #  group_no = 2

    #else:
    #pattern  = "O?\s?SR\.?\s?(DEPUTADO)?\s+JAIR\s+BOLSONARO\s?(\((Bloco\/)?\w{2,}\s?\-\s?\w{2}[^)]+\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    group_no = 4
        
    regexp   = re.compile(pattern, re.MULTILINE)
    matches  = re.finditer(regexp, clean_string)
    
    full_clean_quote    = [ ]
    full_original_quote = [ ]
    
    for match in matches:
                        
        match_start = match.start(group_no)
        match_end   = match.end(group_no)
            
        clean_quote = match[group_no]
        clean_quote = clean_quote.replace("- ", "")
        
        original_quote = original_string[match_start:match_end]
        original_quote = original_quote.replace("- ", "")
        
        full_clean_quote.append(clean_quote)
        full_original_quote.append(original_quote)
        
    full_clean_quote    = ' [ INTERRUPÇÃO ] '.join(full_clean_quote)
    full_original_quote = ' [ INTERRUPÇÃO ] '.join(full_original_quote)

    # Remove espaços múltiplos internos usando a operação join de lista
    full_clean_quote    = ' '.join(full_clean_quote.split()) 
    full_original_quote = ' '.join(full_original_quote.split()) 
    
    return full_clean_quote, full_original_quote

#### Funções para aplicar operações de regex no dataframe

In [7]:
def apply_count_speakers(row):
    
    '''
    Aplica, linha a linha, a função
    find_speakers(string)
    '''
    
    matches = find_speakers(row.CLEAN_CONTENT)
    speaker_count = len(matches)
    
    return pd.Series({ "SPEAKER_COUNT":speaker_count 
        })

In [8]:
def apply_extract_full_quote(row):
    
    '''
    Aplica, linha a linha, a função
    extract_full_quote(string)
    '''
    
    full_clean_quote, full_original_quote = extract_full_quote(row.CLEAN_CONTENT, row.ORIGINAL_CONTENT)
    return pd.Series({
        "FULL_CLEAN_QUOTE"    : full_clean_quote,
        "FULL_ORIGINAL_QUOTE" : full_original_quote
    })

#### Função que encapsula anteriores e roda a operação

In [9]:
def get_speak(zip_file = False):
    '''
    zip_file -> indica se os arquivos estão compactados no formato ZIP.
    
    Executa as operações anteriores em ambos os bancos de dados
    (plenário e comissões), filtra entradas sem match, concatena
    ambos os dataframes e salva para arquivo csv.
    '''
    list_info_file = []
    list_speak = []
    
    if zip_file:
        #list_speak, list_info_file = read_file( find_fpaths("../data/txts/plenario/", "*.zip"), zip_file)
        list_speak, list_info_file = read_file( find_fpaths("/disco01/gabriel/plenario/", "*.zip"), zip_file)
    else:
        list_speak, list_info_file = read_file(find_fpaths("../data/txts/plenario/", "*.txt"),zip_file)
        
    
    return list_speak, list_info_file

In [10]:
def get_metadata(year = None):
    if year:
        metadata = pd.read_csv(path+'plenario/tables/{}-plenario-metadata.csv'.format(year))
        #print(metadata.shape)
        return metadata
    return None

In [11]:
## LENDO OS DISCURSOS E SALVANDO EM UM DATAFRAME
path = '/disco01/gabriel/'
speak, info = get_speak(True)

Reading Files...
/disco01/gabriel/plenario/2001.zip 19355
/disco01/gabriel/plenario/2005.zip 20185
/disco01/gabriel/plenario/2012.zip 20031
/disco01/gabriel/plenario/2002.zip 10353
/disco01/gabriel/plenario/2013.zip 27752
/disco01/gabriel/plenario/2014.zip 17303
/disco01/gabriel/plenario/2010.zip 14961
/disco01/gabriel/plenario/2003.zip 23829
/disco01/gabriel/plenario/2016.zip 23092
/disco01/gabriel/plenario/2019.zip 19506
/disco01/gabriel/plenario/2017.zip 28134
/disco01/gabriel/plenario/2018.zip 16560
/disco01/gabriel/plenario/2009.zip 24499
/disco01/gabriel/plenario/2011.zip 22727
/disco01/gabriel/plenario/2008.zip 20488
/disco01/gabriel/plenario/2007.zip 25116
/disco01/gabriel/plenario/2006.zip 15522
/disco01/gabriel/plenario/2015.zip 28169
/disco01/gabriel/plenario/2004.zip 19028


In [15]:
i = 5
#speak[i],info[i]

In [20]:
df_plen = make_df_plen(speak,info)
#df_plen['INFO'] = info

#As sessões do tipo HOMENAGEM são apenas registro de protocolo. 
#Não contém transcrição de discursos.
df_plen = df_plen[ df_plen.SESSION_TYPE != "HOMENAGEM" ]

# Aplica funções para extrair discursos
df_plen["SPEAKER_COUNT"] = df_plen.apply(apply_count_speakers, axis=1)
df_plen.index = [i for i in range(len(df_plen))]

#return df_plen


Making  DF plen...


In [21]:
len(df_plen)

113143

In [24]:
## inserindo metadados no data frame dos discursos
df_metadata = pd.DataFrame()
for y in df_plen.YEAR.unique():
    print(y)
    metadata = get_metadata(y)
    #if metadata != None:
    for f in df_plen.loc[df_plen.YEAR == y,'FILE'].apply(lambda x: x.split('-')[0]):
        f = int(f)
        df_metadata = pd.concat([df_metadata,metadata.loc[f:f,['Orador','Hora']]])

df_metadata.index = [i for i in range(len(df_metadata))]

df_plen = pd.concat([df_plen,df_metadata],axis=1)
print('Finalizado...')

2001
2005
2012
2002
2013
2014
2010
2018
2003
2016
2019
2017
2009
2011
2008
2007
2006
2015
2004


In [ ]:
df_plen.head(20)

In [25]:
orador = []
partido = []
ufPartido = []

for i,d in enumerate(df_plen.Orador):#[0].split('-')
    splitOrador  = d.split(',')
    #print(i,d,splitOrador)
    
    if len(splitOrador) > 1:
        splitPartido = splitOrador[1].split('-')
        
        if len(splitPartido) < 2:
            partido.append('N/I')
            ufPartido.append('N/I')
        else:
            partido.append(splitPartido[0])
            ufPartido.append(splitPartido[1])
    else:
        splitPartido = splitOrador[0].split('-')
        partido.append('N/I')
        ufPartido.append('N/I')
    
    orador.append(splitOrador[0])
    if len(splitPartido) < 2 or len(splitPartido) > 2:
        print(d,splitPartido)
    
print('Finalizado...')


ROBERTO LEHER(PRESIDENTE DO SINDICATO NACIONAL DOS DOCENTES DAS INSTITUIÇÕES DE ENSINO SUPERIOR) ['ROBERTO LEHER(PRESIDENTE DO SINDICATO NACIONAL DOS DOCENTES DAS INSTITUIÇÕES DE ENSINO SUPERIOR)']
NIVALDO SANTANA (DEPUTADO ESTADUAL DE SÃO PAULO, SP) [' SP)']
FRANCISCO ALMEIDA FILHO (DIRETOR DO DEPARTAMENTO DO PATRIMÔNIO HISTÓRICO E ARTÍSTICO DE BRASÍLIA) ['FRANCISCO ALMEIDA FILHO (DIRETOR DO DEPARTAMENTO DO PATRIMÔNIO HISTÓRICO E ARTÍSTICO DE BRASÍLIA)']
GERMANO LAMBERT TORRES(PRÓ-DIRETOR DE PESQUISA E PÓS-GRADUAÇÃO DA ESCOLA FEDERAL DE ENGENHARIA DE ITAJUBÁ) ['GERMANO LAMBERT TORRES(PRÓ', 'DIRETOR DE PESQUISA E PÓS', 'GRADUAÇÃO DA ESCOLA FEDERAL DE ENGENHARIA DE ITAJUBÁ)']
JOÃO CARLOS CASCAIS(REPRESENTANTE DO CONSELHO REGIONAL DE ENGENHARIA E ARQUITETURA DO PARANÁ) ['JOÃO CARLOS CASCAIS(REPRESENTANTE DO CONSELHO REGIONAL DE ENGENHARIA E ARQUITETURA DO PARANÁ)']
AÉCIO NEVES (PRESIDENTE) ['AÉCIO NEVES (PRESIDENTE)']
DANTE ALARIO JUNIOR ( REPRESENTANTE DO BIOLAB) ['DANTE ALARIO JUNIOR (

MARLENE LIBARDONI - DIRETORA-EXECUTIVA DA AGENDE — AÇÕES EM GÊNERO, CIDADANIA E DESENVOLVIMENTO. [' CIDADANIA E DESENVOLVIMENTO.']
MARCOS JOAQUIM GONÇALVES ALVES-MEMBRO DO CONSELHO DE RELAÇÕES TRABALHISTAS DA ASSOCIAÇÃO BRASILEIRA DA INFRA-ESTRUTURA E INDÚSTRIAS DE BASE - ABDIB ['MARCOS JOAQUIM GONÇALVES ALVES', 'MEMBRO DO CONSELHO DE RELAÇÕES TRABALHISTAS DA ASSOCIAÇÃO BRASILEIRA DA INFRA', 'ESTRUTURA E INDÚSTRIAS DE BASE ', ' ABDIB']
MARISA SANABRIA - PSICÓLOGA, MESTRE EM FILOSOFIA PELA UNIVERSIDADE FEDERAL DE MINAS GERAIS — UFMG E MEMBRO DA ACADEMIA FEMININA MINEIRA DE LETRAS [' MESTRE EM FILOSOFIA PELA UNIVERSIDADE FEDERAL DE MINAS GERAIS — UFMG E MEMBRO DA ACADEMIA FEMININA MINEIRA DE LETRAS']
KÁTIA MARIA BARRETO SOUTO - MEMBRO DA COORDENAÇÃO NACIONAL E CONSELHO CONSULTIVO DA UNIÃO BRASILEIRA DE MULHERES - UBM ['KÁTIA MARIA BARRETO SOUTO ', ' MEMBRO DA COORDENAÇÃO NACIONAL E CONSELHO CONSULTIVO DA UNIÃO BRASILEIRA DE MULHERES ', ' UBM']
NEUSA RISETTE - DIRETORA-EXECUTIVA DA ASSOCI

In [26]:

df_plen['ORADOR_CLEAN'] = orador
df_plen['PARTIDO']      = partido
df_plen['UF_PARTIDO']   = ufPartido


In [31]:
df_plen.SESSION_DATE = df_plen.SESSION_DATE.astype(str)

In [32]:
df_plen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113143 entries, 0 to 113142
Data columns (total 13 columns):
FILE                113143 non-null object
ORIGINAL_CONTENT    113143 non-null object
CLEAN_CONTENT       113143 non-null object
SESSION_TYPE        113143 non-null object
SESSION_DATE        113143 non-null object
MONTH               113143 non-null int64
YEAR                113143 non-null int64
SPEAKER_COUNT       113143 non-null int64
Orador              113143 non-null object
Hora                113143 non-null object
ORADOR_CLEAN        113143 non-null object
PARTIDO             113143 non-null object
UF_PARTIDO          113143 non-null object
dtypes: int64(3), object(10)
memory usage: 17.1+ MB


In [27]:
df_plen.index = [i for i in range(len(df_plen))]
len(df_plen)

113143

In [33]:
# Salva
directory = path #"../data/csvs/"
if not os.path.exists(directory):
    os.makedirs(directory)

#name_file = directory + 'discursos_plen.csv'#str(df.YEAR.unique()[0]) + '_plen.csv'
#df_plen.to_csv(name_file, index = False)

name_file = directory + 'discursos_plen.json'#str(df.YEAR.unique()[0]) + '_plen.csv'
df_plen.to_json(name_file)


In [ ]:
df_metadata
d.split(',')
pd.Series(ufPartido).value_counts()
#metadata.loc[int(f):int(f),['Orador','Hora']]

In [ ]:
df_plen.head(10)
#len(df_plen)
#metadata.loc[f:int(f),['Orador','Hora']]

In [ ]:
df_plen.index = [i for i in range(len(df_plen))]
df_plen

In [ ]:
a = pd.DataFrame([1,2,3])
b = pd.DataFrame([10,11,12])

pd.concat([a,b],axis = 1)

In [ ]:
metadata.iloc[0]

In [ ]:
print('Qtd de Registros por ANO:\n{}\n\nQtd Total de registros:{} '.format(df_plen.YEAR.value_counts(),len(df_plen)))

In [ ]:
i = 0
session_date = []
for file in info:
    #print(i)
    result = re.search("\-([A-Z\s]+)\-", file)# info[108251])
    if result is None:
        session_date.append('none')
    else:
        session_date.append(result.group(1))
    #'SESSION_DATE'     : [ re.search("\d{8}", file).group(0) for file in content ]
    i += 1

In [ ]:
len(session_date)
len(info)